In [1]:
import sys
sys.setrecursionlimit(1000000000)
from copy import deepcopy

In [2]:
with open('input.txt') as fl:
    rules = {}
    for ln in fl.readlines():
        ln = ln.strip()
        if len(ln)>0:
            name, rule = ln.strip('}').split('{')
            rule = rule.split(',')
            rule = [cond.split(':') for cond in rule]
            rules[name] = rule
        else:
            break

In [3]:
#rules

In [4]:
def flip_cond(cond):
    return cond.replace('>','LE').replace('<','GE').replace('LE','<=').replace('GE','>=')

In [5]:
def update_xmas(xmas, rule):
    xmas = deepcopy(xmas)
    for cond in rule:
        v = cond[0]
        num = int(cond[1:].strip('<>='))
        if '>' in cond:
            if '=' in cond:
                num -= 1
            xmas[v][0] = max(num+1, xmas[v][0])
        elif '<' in cond:
            if '=' in cond:
                num += 1
            xmas[v][1] = min(num-1, xmas[v][1])    
    return xmas

def validate_xmas(xmas):
    return all(v[0]<=v[1] for v in xmas.values())

In [6]:
def solve_xmas(xmass=[('in',{'x':[1,4000], 'm':[1,4000], 'a':[1,4000], 's':[1,4000]})]):

    new_xmass = []
    for start, xmas in xmass:
        conds = deepcopy(rules[start])
        current_cases = []
        for cond, target in conds[:-1]:
            current_cases.append(cond)
            new_xmas = update_xmas(xmas, current_cases)
            new_xmass.append((target, new_xmas))
            current_cases[-1] = flip_cond(current_cases[-1])
        new_xmass.append((conds[-1][0], update_xmas(xmas, current_cases)))
        
    solved = [(start, xmas) for start, xmas in new_xmass if start in ('A','R')]
    unsolved = [(start, xmas) for start, xmas in new_xmass if start not in ('A','R')]
    
    if len(unsolved)>0:
        solved.extend(solve_xmas(unsolved))
    
    return solved
        


In [7]:
score = 0
for xmas in [xmas for start, xmas in solve_xmas() if start == 'A']:
    v = 1
    for lb, ub in xmas.values():
        v *= ub - lb + 1
    score += v
score

116138474394508